In [1]:
from factor_processing_utils import *
import matplotlib.pyplot as plt

In [2]:
start_date = "2020-01-01"
end_date = "2025-07-01"
index_item = "000852.XSHG"
change_days = [5, 10, 20]

stock_universe = INDEX_FIX(start_date, end_date, index_item)
stock_list = stock_universe.columns.tolist()
date_list = stock_universe.index.tolist()


factor_name = "cfoa_mrq"
factor_definition = Factor("cash_flow_from_operating_activities_mrq_0") / Factor(
    "total_assets_mrq_0"
)
factor_raw = execute_factor(factor_definition, stock_list, start_date, end_date)
factor_raw.to_pickle(
    f"2025-08-07/{factor_name}_{index_item}_{start_date}_{end_date}.pkl"
)
# 数据清洗
factor_clean = data_clean(factor_raw, stock_universe, index_item)
# 计算IC
for change_day in change_days:
    ic, performance = calc_ic(factor_clean, change_day, index_item, factor_name)

{'name': 'cfoa_mrq', 'change_day': 5, 'IC mean': 0.0145, 'IC std': 0.0464, 'IR': 0.3113, 'IC>0': 0.6163, 'ABS_IC>2%': 0.6835, 't_stat': 11.3267}
{'name': 'cfoa_mrq', 'change_day': 10, 'IC mean': 0.0176, 'IC std': 0.0488, 'IR': 0.3618, 'IC>0': 0.6164, 'ABS_IC>2%': 0.6725, 't_stat': 13.1383}
{'name': 'cfoa_mrq', 'change_day': 20, 'IC mean': 0.0225, 'IC std': 0.0542, 'IR': 0.4148, 'IC>0': 0.6425, 'ABS_IC>2%': 0.7036, 't_stat': 15.0059}


In [ ]:
group_return, turnover_ratio = group_g(
    factor_clean, change_day, 10, index_item, name=factor_name
)